In [ ]:
# %pip install -q git+https://github.com/pyannote/pyannote-audio > /dev/null

In [148]:
import os
import concurrent.futures
import json
import hashlib
from dotenv import load_dotenv
from openai import OpenAI
# from deepgram import (
#     DeepgramClient,
#     PrerecordedOptions,
#     FileSource,
# )
import pyannote.audio
load_dotenv()

True

In [149]:
# Set up clients
dz_client = DeepgramClient()
dz_client.api_key = os.getenv("DEEPGRAM_API_KEY")

tr_client = OpenAI()
tr_client.api_key = os.getenv("OPENAI_API_KEY")

In [150]:
# Choose file and languages
file_path = 'uploads/ar-en-audio-short.mp3'
languages = ['en','ar']

In [151]:
CACHE_FILE = 'cache.json'    
file_hash = hashlib.md5(open(file_path, 'rb').read()).hexdigest()

def load_cache():
    try:
        with open(CACHE_FILE, 'r') as file:
            return json.load(file)
    except (FileNotFoundError, json.JSONDecodeError):
        return {}  # Return an empty dictionary if the file doesn't exist or if an error occurs

def save_cache(cache):
    with open(CACHE_FILE, 'w') as file:
        json.dump(cache, file, indent=4, ensure_ascii=False)
        
cache = load_cache()
print(cache.keys())

dict_keys(['0fb08695744589d8a434246f5d347204_en', '0fb08695744589d8a434246f5d347204_ar'])


In [ ]:
# Local diarization with Pyannote.
def diarize(audio_file_path, file_hash):
    return None

In [152]:
# API diarization with Deepgram.
# def diarize(audio_file_path, file_hash):
#     cache_key = f"{file_hash}_diarize"
#     if cache_key in cache:
#         return cache[cache_key]
#     with open(audio_file_path, "rb") as file:
#         buffer_data = file.read()
#     payload: FileSource = {
#         "buffer": buffer_data,
#     }
#     options = PrerecordedOptions(
#         model="base",
#         diarize=True,
#         detect_language=True
#     )
#     response = dz_client.listen.prerecorded.v("1").transcribe_file(payload, options)


#     # Parse the response JSON
#     response_json = json.loads(response.to_json())
    
#     cache[cache_key] = response_json
#     save_cache(cache)
    
#     # Format the JSON with proper indentation when returning
#     return response_json


In [153]:
def transcribe(audio_file_path, file_hash, language: str):
    cache_key = f"{file_hash}_{language}"
    if cache_key in cache:
        return cache[cache_key]
    with open(audio_file_path, "rb") as audio_file:
        response = tr_client.audio.transcriptions.create(
            model="whisper-1",
            language=language,
            file=audio_file,
            response_format="verbose_json",
            timestamp_granularities=["word", "segment"]
        )
    response_json = response.model_dump()  # Convert to dictionary
    cache[cache_key] = response_json
    save_cache(cache)
    
    return response_json


In [154]:
def process_audio_file(file_path, languages: list[str]):

    with concurrent.futures.ThreadPoolExecutor() as executor:
        future_transcriptions = {
            lang: executor.submit(transcribe, file_path, file_hash, lang)
            for lang in languages
        }
        transcriptions = {
            lang: future.result()
            for lang, future in future_transcriptions.items()
        }
    
        future_diarization = executor.submit(diarize, file_path, file_hash)
        diarization = future_diarization.result()

    return diarization, transcriptions

In [155]:
diarization, transcriptions = process_audio_file(file_path, languages)

In [146]:
print("Diarization:")
print(json.dumps(diarization, indent=4))

print("\nTranscriptions:")
for lang, transcription in transcriptions.items():
    print(f"{lang} Transcript:")
    print(transcription)

Diarization:
{
    "metadata": {
        "transaction_key": "deprecated",
        "request_id": "b3a00b1d-32f2-4315-bad4-90c0600f2b5c",
        "sha256": "969fdef902ad45072d37f2ad9598394fc5cdb32d5f07606d6dfaa73ed026a637",
        "created": "2024-03-28T21:18:25.620Z",
        "duration": 390.99576,
        "channels": 1,
        "models": [
            "1ed36bac-f71c-4f3f-a31f-02fd6525c489"
        ],
        "model_info": {
            "1ed36bac-f71c-4f3f-a31f-02fd6525c489": {
                "name": "general",
                "version": "2024-01-26.8851",
                "arch": "base"
            }
        }
    },
    "results": {
        "channels": [
            {
                "alternatives": [
                    {
                        "transcript": "okay so then if we can start again i'm my cool jim next is so what does the economic office in particular have to offer to women's in area should so in general women in our society were deprived of having access to the right t